In [1]:
import pandas as pd
from auto_matcher.utils import reduce_and_add_match


FP_ZERO_SHOT_RESULTS = "../playground/zero_shot_ops.csv"
FP_HISTORICAL_MATCHING = "../historical_data/all_to_OMOP_Mapping.csv"

In [2]:
df_history = pd.read_csv(FP_HISTORICAL_MATCHING)
df_pred = pd.read_csv(FP_ZERO_SHOT_RESULTS)

In [3]:
df_history.head()

,source_table,source_column,target_table,target_column
0,ADMISSIONS,SUBJECT_ID,VISIT_OCCURRENCE,person_id
1,ADMISSIONS,HADM_ID,VISIT_OCCURRENCE,visit_occurrence_id
2,ADMISSIONS,ADMITTIME,VISIT_OCCURRENCE,visit_start_datetime
3,ADMISSIONS,DISCHTIME,VISIT_OCCURRENCE,visit_end_datetime
4,ADMISSIONS,DEATHTIME,NaN,NaN


In [4]:
df_pred.head()

,source_table,source_column,target_table_pred,target_table_column_pred
0,ADMISSIONS,SUBJECT_ID,person,person_id
1,ADMISSIONS,HADM_ID,visit_occurrence,visit_occurrence_id
2,ADMISSIONS,ADMITTIME,VISIT_OCCURRENCE,visit_start_datetime
3,ADMISSIONS,DISCHTIME,VISIT_OCCURRENCE,visit_end_datetime
4,ADMISSIONS,DEATHTIME,DEATH,death_datetime


In [5]:
df_history.columns = [
    "source_table",
    "source_column",
    "target_table_true",
    "target_column_true",
]
df_pred.columns = [
    "source_table",
    "source_column",
    "target_table_pred",
    "target_column_pred",
]
df_match = df_history.merge(
    df_pred,
    on=["source_table", "source_column"],
    how="left",
    suffixes=("_true", "_pred"),
).fillna("none")

In [6]:
df_match.head()

,source_table,source_column,target_table_true,target_column_true,target_table_pred,target_column_pred
0,ADMISSIONS,SUBJECT_ID,VISIT_OCCURRENCE,person_id,person,person_id
1,ADMISSIONS,HADM_ID,VISIT_OCCURRENCE,visit_occurrence_id,visit_occurrence,visit_occurrence_id
2,ADMISSIONS,ADMITTIME,VISIT_OCCURRENCE,visit_start_datetime,VISIT_OCCURRENCE,visit_start_datetime
3,ADMISSIONS,DISCHTIME,VISIT_OCCURRENCE,visit_end_datetime,VISIT_OCCURRENCE,visit_end_datetime
4,ADMISSIONS,DEATHTIME,none,none,DEATH,death_datetime


In [7]:
df_ouput = reduce_and_add_match(df_match, with_confidence=False)

2024-09-03 08:59:22.560 | SUCCESS  | auto_matcher.utils:reduce_and_add_match:126 - ValueCounts for Types:{'Type2': 194, 'Type5': 138, 'Type1': 92, 'Type4': 43, 'Type3': 36}


In [12]:
df_ouput.match_type.value_counts().sort_index()

match_type
Type1     92
Type2    194
Type3     36
Type4     43
Type5    138
Name: count, dtype: int64